<a href="https://colab.research.google.com/github/Rustydinoco/ML_Learning/blob/master/Streamlit_LSTM_S%26P_500_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q streamlit pyngrok

In [17]:
import joblib
from sklearn.preprocessing import MinMaxScaler

joblib.dump(MinMaxScaler, 'MinMaxScaler.joblib')


['MinMaxScaler.joblib']

In [18]:
%%writefile S&P500.py
import torch
import torch.nn as nn
import torch.nn.functional as f
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import streamlit as st
from pyngrok import ngrok
import getpass

class LSTMModel(nn.Module):
  def __init__(self, input_size = 1, hidden_layer_size  = 100, output= 1):
    super().__init__()
    self.hidden_layer_size = hidden_layer_size
    self.lstm = nn.LSTM(input_size, hidden_layer_size)
    self.linear = nn.Linear(hidden_layer_size, output)

  def forward(self, input_seq):
    lstm_out, _ = self.lstm(input_seq)
    prediction = self.linear(lstm_out[:, -1, :])
    return prediction


@st.cache_resource
def load_data(ticker):
  df = yf.download(ticker, start="2010-01-01")
  return df

@st.cache_resource
def load_model_and_scaler():
  try:
      model = LSTMModel()
      model.load_state_dict(torch.load('model_LSTM_S&P500.pth'))
      model.eval()

      scaler = joblib.load('MinMaxScaler.joblib')
      return model, scaler
  except Exception as e:
      st.error("File Not Found")
      return None, None


st.title("S&P 500 Stock Prediction")

data = load_data("^GSPC")
model, scaler = load_model_and_scaler()

if model is not None:
  st.subheader("Close Price S&P 500")
  fig = go.Figure()
  fig.add_trace(go.Scatter(x= data["Date"], y= data["Close"], name= "S&P 500"))
  fig.layout.update(title_text = "S&P 500 Price Movement", xaxis_rangeslider_visible = True)
  st.plotly_chart(fig)

  st.sidebar.header("Prediksi Harga")

  if st.sidebar.button("Prediksi Harga Selanjutnya"):
    look_back = 60
    last_60_days = data["Close"].values[-look_back:]
    last_60_days_scaled = scaler.transform(last_60_days.reshape(-1,1))
    X_pred = torch.FloatTensor(last_60_days_scaled)

    with torch.no_grad:
      prediction_scaled = model(X_pred)

    prediction_actual = scaler.inverse_transform(prediction_scaled.numpy().reshape(-1,1))

    last_date = data["Date"].iloc[-1]
    next_date = last_date + pd.Timedelta(days= 1)

    st.sidebar.metric(
            label=f"Prediksi Harga Tutup untuk {next_date.strftime('%Y-%m-%d')}",
            value=f"${prediction_actual[0][0]:,.2f}"
   )
st.sidebar.info("Disclaimer: Ini adalah proyek teknis, bukan saran finansial.")

Overwriting S&P500.py


In [19]:
!ngrok authtoken 2zH8hZqsfmzIXSAGlYPcQjgFkL8_75gw42YGnjStbTWgoxPJd

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
!streamlit run "S&P500.py" --server.port 8051

public_url = ngrok.connect(8051)
print(f"Silahkan Klik Link Ini {public_url}")




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8051
  Network URL: http://172.28.0.12:8051
  External URL: http://35.194.154.229:8051

/content/S&P500.py:27: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2010-01-01")
[*********************100%***********************]  1 of 1 completed
